# **1. Thu thập dữ liệu**




**Cách thức thu thập**

Nguồn dữ liệu: trang báo điện tử VNExpress (https://vnexpress.net), Dân trí (https://dantri.com.vn), Tuổi trẻ (https://tuoitre.vn/).

**Quy trình thu thập:**
*   Viết script Python để truy cập từng chuyên mục trên các trang web.
*   Sử dụng BeautifulSoup để trích xuất các thẻ HTML chứa thông tin cần thiết,
dùng Selenium để cuộn trang.
*  Lưu dữ liệu vào file CSV với các cột: title, description, content, category, author, link, category.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Crawl nguồn vnexpress

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Danh mục cần crawl
categories = {
    'suc-khoe': 'Sức khỏe',
    'phap-luat': 'Pháp luật',
    'kinh-doanh': 'Kinh doanh',
    'khoa-hoc': 'Khoa học',
    'the-thao': 'Thể thao',
    'giai-tri': 'Giải trí',
    'giao-duc': 'Giáo dục',
    'du-lich': 'Du lịch'
}

# Hàm crawl nội dung chi tiết của một bài báo
def crawl_article_content(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Lấy nội dung bài báo
        content_tag = soup.find('article', class_='fck_detail')
        content = content_tag.get_text(strip=True) if content_tag else "Không tìm thấy nội dung"

        # Lấy tác giả
        author = "Không rõ tác giả"

        # Tìm trong phần nội dung bài báo
        if content_tag:
            strong_tags = content_tag.find_all('strong')
            if strong_tags:
                last_strong = strong_tags[-1]
                text = last_strong.get_text(strip=True)
                if text:
                    author_match = re.match(r'^(.*?)(?:\s*\(Theo.*\))?$', text)
                    if author_match:
                        author = author_match.group(1).strip()
                        if not author:
                            author = "Không rõ tác giả"

        # Nếu không tìm thấy trong <article>, thử tìm trong <p class="author_mail">
        if author == "Không rõ tác giả":
            author_mail_tag = soup.find('p', class_='author_mail')
            if author_mail_tag:
                strong_tag = author_mail_tag.find('strong')
                if strong_tag:
                    text = strong_tag.get_text(strip=True)
                    author_match = re.match(r'^(.*?)(?:\s*\(Theo.*\))?$', text)
                    author = author_match.group(1).strip() if author_match else text

        return content, author

    except Exception as e:
        print(f"Lỗi khi crawl nội dung bài báo {url}: {e}")
        return "Lỗi khi crawl nội dung", "Không rõ tác giả"

# Hàm crawl một trang trong danh mục
def crawl_page(url, category_name):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Tìm các bài báo
        articles = soup.find_all('h3', class_='title-news')
        if not articles:
            print(f"Không tìm thấy bài báo nào trên trang {url}")
            return []

        data = []
        for article in articles:
            title_tag = article.find('a')
            if not title_tag:
                continue
            title = title_tag.text.strip()
            link = title_tag['href']
            if not link.startswith('https'):
                link = 'https://vnexpress.net' + link

            description_tag = article.find_next('p', class_='description')
            description = description_tag.text.strip() if description_tag else "Không có mô tả"

            content, author = crawl_article_content(link)

            data.append({
                'title': title,
                'description': description,
                'content': content,
                'author': author,
                'link': link,
                'category': category_name
            })

            time.sleep(1)

        return data
    except Exception as e:
        print(f"Lỗi khi crawl trang {url}: {e}")
        return []

# Hàm crawl toàn bộ danh mục
def crawl_category(category_key, category_name, max_articles=500):
    base_url = f"https://vnexpress.net/{category_key}"
    articles_per_page = 25
    pages_to_crawl = (max_articles // articles_per_page) + 1
    if pages_to_crawl > 12:
        pages_to_crawl = 12

    all_data = []
    for page in range(1, pages_to_crawl + 1):
        if page == 1:
            url = base_url
        else:
            url = f"{base_url}-p{page}"

        print(f"Đang crawl trang {page} của danh mục {category_name}: {url}")
        page_data = crawl_page(url, category_name)
        all_data.extend(page_data)

        if len(all_data) >= max_articles:
            all_data = all_data[:max_articles]
            break

        time.sleep(2)

    return all_data

# Hàm lưu dữ liệu vào file CSV
def save_to_csv(data, filename="vnexpress_articles.csv"):
    if not data:
        print("Không có dữ liệu để lưu.")
        return

    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Dữ liệu đã được lưu vào {filename}")

# Hàm chính
def main():
    all_articles = []
    for category_key, category_name in categories.items():
        print(f"\nBắt đầu crawl danh mục: {category_name}")
        category_data = crawl_category(category_key, category_name, max_articles=600)
        all_articles.extend(category_data)

        save_to_csv(category_data, f"vnexpress_{category_key}.csv")

    save_to_csv(all_articles, "/content/drive/MyDrive/FILE/Năm 3/Khoa học dữ liệu/Nhóm 09 - PTDL Báo điện tử/vnexpress_all_articles.csv")

if __name__ == "__main__":
    main()


Bắt đầu crawl danh mục: Sức khỏe
Đang crawl trang 1 của danh mục Sức khỏe: https://vnexpress.net/suc-khoe
Đang crawl trang 2 của danh mục Sức khỏe: https://vnexpress.net/suc-khoe-p2
Không tìm thấy bài báo nào trên trang https://vnexpress.net/suc-khoe-p2
Đang crawl trang 3 của danh mục Sức khỏe: https://vnexpress.net/suc-khoe-p3
Không tìm thấy bài báo nào trên trang https://vnexpress.net/suc-khoe-p3
Đang crawl trang 4 của danh mục Sức khỏe: https://vnexpress.net/suc-khoe-p4
Không tìm thấy bài báo nào trên trang https://vnexpress.net/suc-khoe-p4
Đang crawl trang 5 của danh mục Sức khỏe: https://vnexpress.net/suc-khoe-p5
Không tìm thấy bài báo nào trên trang https://vnexpress.net/suc-khoe-p5
Đang crawl trang 6 của danh mục Sức khỏe: https://vnexpress.net/suc-khoe-p6
Không tìm thấy bài báo nào trên trang https://vnexpress.net/suc-khoe-p6
Đang crawl trang 7 của danh mục Sức khỏe: https://vnexpress.net/suc-khoe-p7
Không tìm thấy bài báo nào trên trang https://vnexpress.net/suc-khoe-p7
Đang 

# Crawl Nguồn Dân trí

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
# Không cần import Selenium cho Dân Trí trong trường hợp này
# vì hàm tìm link dùng requests đã hoạt động tốt
import csv
import re

# --- Hàm trợ giúp chung (Requests) ---
def fetch_page_details(url):
    """Gửi yêu cầu HTTP GET và phân tích HTML để trích xuất chi tiết."""
    headers = {
        # Sử dụng User-Agent gần đây hơn
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers, timeout=25) # Timeout hợp lý
        response.raise_for_status()
        response.encoding = 'utf-8' # Đảm bảo encoding UTF-8
        return BeautifulSoup(response.text, 'html.parser')
    except requests.Timeout:
        print(f"Timeout khi tải trang chi tiết {url}")
        return None
    except requests.RequestException as e:
        print(f"Lỗi requests khi tải trang chi tiết {url}: {e}")
        return None
    except Exception as e:
        print(f"Lỗi không xác định khi tải {url}: {e}")
        return None

# --- Hàm tìm link Dân Trí (Giữ nguyên từ code của bạn) ---
def crawl_dantri_links(category, number_target=1000):
    """Thu thập link bài báo từ các trang danh mục Dân Trí bằng Requests."""
    base_url = f"https://dantri.com.vn/{category}.htm"
    links = set()
    page = 1
    max_retries = 3
    retries = 0
    max_pages_to_check = 100 # Giới hạn số trang kiểm tra để tránh vòng lặp vô hạn

    print(f"Bắt đầu thu thập link Dân Trí: {base_url} (Mục tiêu: {number_target})")

    while len(links) < number_target and page <= max_pages_to_check:
        url = base_url if page == 1 else f"https://dantri.com.vn/{category}/trang-{page}.htm"
        print(f"   Đang quét trang: {url} (Đã tìm thấy: {len(links)}/{number_target})")

        # Sử dụng hàm fetch_page_details để lấy soup
        soup = fetch_page_details(url)

        if not soup:
            retries += 1
            print(f"   Không thể tải trang {url}, thử lại {retries}/{max_retries}...")
            if retries >= max_retries:
                print(f"   Bỏ qua trang {url} sau {max_retries} lần thử thất bại.")
                page += 1 # Chuyển sang trang tiếp theo nếu lỗi
                retries = 0
                continue
            time.sleep(5) # Chờ lâu hơn trước khi thử lại
            continue

        retries = 0 # Reset retries nếu thành công

        # Tìm các thẻ chứa link bài báo (Cập nhật selector nếu cần)
        # Thử nhiều selector phổ biến trên Dân Trí
        article_containers = soup.select('article.article-item h3.article-title a, div.article-item h3.article-title a, h2.news-item__title a')
        found_new_on_page = False

        if not article_containers:
             print(f"   Không tìm thấy thẻ bài báo nào trên trang {page}. Có thể đã hết trang hoặc cấu trúc thay đổi.")
             break # Dừng nếu không tìm thấy bài nào trên trang

        for a_tag in article_containers:
            if a_tag and 'href' in a_tag.attrs:
                link = a_tag['href']
                # Chuẩn hóa link (thêm domain nếu thiếu)
                if not link.startswith('http'):
                    link = f"https://dantri.com.vn{link}"

                # Kiểm tra xem link có vẻ hợp lệ và thuộc danh mục không
                # Điều kiện f"/{category}/" trong link có thể quá chặt, xem xét lại nếu cần
                # Ví dụ: Chỉ cần link bắt đầu bằng domain và có định dạng .htm
                if link.startswith('https://dantri.com.vn/') and link.endswith('.htm') and link not in links:
                     # Thêm bộ lọc loại trừ video, chủ đề đặc biệt nếu cần
                     if "/video-" not in link and "/topic/" not in link:
                          links.add(link)
                          found_new_on_page = True
                          # print(f"      + Link mới: {link}") # Debug
                          if len(links) >= number_target:
                              break # Đủ số lượng mục tiêu

        if len(links) >= number_target:
            print(f"   Đã đạt mục tiêu {number_target} link.")
            break # Thoát vòng lặp while

        # Nếu trang có bài viết nhưng không tìm thấy link *mới* nào, có thể đã hết bài mới
        if not found_new_on_page and article_containers:
             print(f"   Không tìm thấy link mới nào trên trang {page}. Dừng tìm kiếm cho danh mục này.")
             break

        page += 1
        time.sleep(1.5) # Nghỉ giữa các trang

    print(f"--- Hoàn tất tìm kiếm link cho [{category}]. Tìm thấy tổng cộng {len(links)} link duy nhất. ---")
    return links

# --- Hàm trích xuất chi tiết bài báo Dân Trí (Đã bỏ print xử lý link) ---
def extract_dantri_article_details(url, category):
    """Trích xuất chi tiết bài báo Dân Trí."""
    # print(f"   Đang trích xuất chi tiết từ: {url}") # <--- ĐÃ XÓA
    soup = fetch_page_details(url)
    if not soup:
        return None

    data = {
        "title": "Không tìm thấy tiêu đề",
        "description": "Không tìm thấy mô tả",
        "content": "Không tìm thấy nội dung",
        "author": "Không tìm thấy tác giả",
        "link": url,
        "category": category
    }

    try:
        title_element = soup.find("h1", class_=re.compile(r'\b(dt-news__title|article-title|title-page|detail)\b')) # Thêm title-page, detail
        if title_element: data["title"] = title_element.get_text(strip=True)

        desc_element = soup.find(['h2', 'p'], class_=re.compile(r'\b(dt-news__sapo|singular-sapo|sapo)\b'))
        if desc_element: data["description"] = desc_element.get_text(strip=True)

        author_tag_b = soup.select_one('div.author-wrap div.author-name a b')
        if author_tag_b:
            data["author"] = author_tag_b.get_text(strip=True)
        else:
             author_element = soup.select_one('div.author-name p strong, p.author strong, div.author-name, p.author, .author') # Thêm .author
             if author_element:
                author_text = author_element.get_text(strip=True)
                author_text = re.sub(r'\s*\(.*?\)|\s*\[.*?\]|\s*Theo\s*|PV\s*-\s*|PV$', '', author_text, flags=re.IGNORECASE).strip()
                if author_text and len(author_text) < 100:
                    data["author"] = author_text
                elif soup.find(string=re.compile(r'Theo\s+')):
                    author_match = soup.find(string=re.compile(r'Theo\s+'))
                    if author_match and len(author_match.strip()) < 100:
                         data["author"] = author_match.strip()

        content_div = soup.find("div", class_=re.compile(r'\b(singular-content|dt-news__content)\b'))
        if content_div:
            elements_to_remove_selectors = [
                'figure', 'table', 'div.video', 'div.audio', '.article-related', '.article-event',
                'ul.related-topic', 'aside.article-topic', 'div.adv', 'div[id^="ads"]',
                'div[class*="ads"]', 'div.ecom-box', 'div.author-wrap', 'div.author-name',
                'p.author', '.singular-source', 'div.article-tags', 'div.tags-container',
                'div.bottom-article-action', 'div.social-share', '.like-share-bottom', '.social-pin',
                'blockquote', 'iframe', 'script', 'style', '#reference-articles',
                '.dantri-widget', 'p > strong:contains(">>>")', '.content-box', '.singular-intro',
                'div.toolbar-content', '.controlbar'
            ]
            for selector in elements_to_remove_selectors:
                for unwanted in content_div.select(selector):
                     unwanted.decompose()

            paragraphs = content_div.find_all('p', recursive=False)
            content_parts = []
            if not paragraphs:
                paragraphs = content_div.find_all('p')

            for p in paragraphs:
                 if p.find_parent('figcaption'): continue
                 text = p.get_text(strip=True)
                 if text and len(text) > 15 and not re.match(r'^(Ảnh|Nguồn|Đồ họa|Theo)\s*:', text, re.IGNORECASE):
                     content_parts.append(text)
            clean_content = "\n\n".join(content_parts)

            if not clean_content or len(clean_content) < 100:
                 # print("   Nội dung từ <p> trống hoặc quá ngắn, thử fallback...") # <--- ĐÃ XÓA
                 clean_content_fallback = content_div.get_text(separator='\n', strip=True)
                 lines = [line.strip() for line in clean_content_fallback.splitlines() if line.strip()]
                 filtered_lines = []
                 for line in lines:
                      if len(line) > 15 and not re.match(r'^(Ảnh|Nguồn|Đồ họa|Theo)\s*:|\(Dân trí\)|Thứ \w+,|Bài viết cùng tác giả|Tin liên quan|Dòng sự kiện|Từ khoá:|Bình luận|Video:|Bạn nghĩ gì|Gửi bình luận|Quan tâm nhất|Mới nhất|Trả lời|Thích|Đọc thêm', line, re.IGNORECASE) and "Ảnh:" not in line and "Clip:" not in line:
                           filtered_lines.append(line)
                 clean_content = "\n\n".join(filtered_lines)
            data["content"] = clean_content.strip()
        return data
    except Exception as e:
        print(f"   Lỗi khi xử lý chi tiết bài báo Dân Trí {url}: {e}") # Giữ lại báo lỗi
        return data

# --- Thực thi chính ---
def main():
    # Cập nhật danh sách danh mục Dân Trí nếu cần
    dantri_categories = [
        "kinh-doanh", "the-thao", "thoi-su", "suc-khoe", "giai-tri",
        "khoa-hoc", "the-gioi", "giao-duc"
    ]
    number_target_per_category = 600 # Mục tiêu số bài viết mỗi danh mục
    all_articles_data = []
    collected_links = set() # Theo dõi link đã xử lý

    print("\n" + "="*10 + f" BẮT ĐẦU THU THẬP DỮ LIỆU DÂN TRÍ (Mục tiêu: {number_target_per_category} bài/danh mục) " + "="*10)
    overall_start_time = time.time()

    for category in dantri_categories:
        print(f"\n===>>> Đang xử lý Danh mục: [{category}] <<<===")
        category_start_time = time.time()
        articles_added_in_category = 0

        # --- Bước 1: Tìm kiếm Links bằng Requests (Hàm của bạn) ---
        article_links_found = crawl_dantri_links(category, number_target_per_category)

        # --- Bước 2: Trích xuất chi tiết cho từng link ---
        if article_links_found:
            print(f"\n---> Bắt đầu trích xuất chi tiết cho {len(article_links_found)} link tìm thấy trong [{category}]...")
            links_to_process = list(article_links_found)

            for i, link in enumerate(links_to_process):
                print(f"   [{category} - {i+1}/{len(links_to_process)}] ", end="")

                if link in collected_links:
                    print(f"Link đã xử lý: {link}. Bỏ qua.")
                    continue

                article_details = extract_dantri_article_details(link, category)

                if article_details:
                    # Kiểm tra tính hợp lệ cơ bản
                    if article_details["title"] != "Không tìm thấy tiêu đề" and \
                       article_details["content"] != "Không tìm thấy nội dung" and \
                       len(article_details["content"]) > 50: # Yêu cầu độ dài nội dung tối thiểu (có thể điều chỉnh)
                        all_articles_data.append(article_details)
                        collected_links.add(link)
                        articles_added_in_category += 1
                        print(f"   => OK. (Đã thêm {articles_added_in_category} bài mới cho danh mục)")
                    else:
                        print(f"   Bài báo không hợp lệ (thiếu TT/ND hoặc ND quá ngắn): {link}. Bỏ qua.")
                else:
                    print(f"   Trích xuất thất bại: {link}. Bỏ qua.")

                time.sleep(0.7) # Tăng delay một chút cho Dân Trí
            print(f"---> Hoàn tất trích xuất cho [{category}]. Đã thêm {articles_added_in_category} bài báo mới.")
        else:
             print(f"!!! Không tìm thấy link nào cho danh mục: [{category}] hoặc đã dừng tìm kiếm.")


        category_end_time = time.time()
        print(f"Thời gian xử lý danh mục [{category}]: {category_end_time - category_start_time:.2f} giây.")
        print("-" * 50)
        # time.sleep(5) # Delay giữa các danh mục nếu cần

    overall_end_time = time.time()
    print("\n" + "="*25)
    print(f"*** HOÀN TẤT TOÀN BỘ QUÁ TRÌNH THU THẬP DỮ LIỆU DÂN TRÍ ***")
    print(f"*** Tổng số bài báo duy nhất đã trích xuất và hợp lệ: {len(all_articles_data)} ***")
    total_minutes = (overall_end_time - overall_start_time) / 60
    print(f"Tổng thời gian thực thi: {total_minutes:.2f} phút")
    print("="*25)

    # --- Bước 3: Lưu dữ liệu vào file CSV ---
    if all_articles_data:
        csv_filepath = '/content/drive/MyDrive/FILE/Năm 3/Khoa học dữ liệu/Nhóm 09 - PTDL Báo điện tử/dantri_articles_data.csv' # Tên file CSV
        fieldnames = ['category', 'link', 'title', 'author', 'description', 'content'] # Thứ tự cột

        print(f"\nĐang lưu dữ liệu vào file CSV: {csv_filepath} ...")
        try:
            with open(csv_filepath, 'w', newline='', encoding='utf-8-sig') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
                writer.writeheader()
                writer.writerows(all_articles_data)
            print(f"===> Đã lưu dữ liệu thành công vào {csv_filepath}")
        except IOError as e:
            print(f"\n!!! Lỗi IO khi lưu file CSV: {e}")
        except Exception as e:
            print(f"\n!!! Lỗi không xác định khi lưu file CSV: {e}")
    else:
        print("\nKhông có dữ liệu bài báo nào hợp lệ để lưu vào file CSV.")

if __name__ == "__main__":
    main()

# Crawl nguồn báo Tuổi trẻ

In [ ]:
pip install selenium

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.chrome.service import Service # Không cần Service nữa
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException, WebDriverException # Thêm WebDriverException
import csv
import re

# --- Hàm trợ giúp (Sử dụng Requests) ---
def fetch_page_details(url):
    """Gửi yêu cầu HTTP GET và phân tích HTML để trích xuất chi tiết."""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        response.encoding = 'utf-8'
        return BeautifulSoup(response.text, 'html.parser')
    except requests.Timeout:
        print(f"Timeout khi tải trang chi tiết {url}")
        return None
    except requests.RequestException as e:
        print(f"Lỗi requests khi tải trang chi tiết {url}: {e}")
        return None
    except Exception as e:
        print(f"Lỗi không xác định khi tải {url}: {e}")
        return None

# --- Hàm trích xuất chi tiết bài báo Tuổi Trẻ ---
def extract_tuoitre_article_details(url, category):
    """
    Trích xuất tiêu đề, mô tả, nội dung, tác giả, link và danh mục
    từ một URL bài báo Tuổi Trẻ.
    """
    print(f"   Đang trích xuất chi tiết từ: {url}")
    soup = fetch_page_details(url)
    if not soup:
        return None

    data = {
        "title": "Không tìm thấy tiêu đề",
        "description": "Không tìm thấy mô tả",
        "content": "Không tìm thấy nội dung",
        "author": "Không tìm thấy tác giả",
        "link": url,
        "category": category
    }

    try:
        # Tiêu đề
        title_element = soup.find("h1", class_=re.compile(r'\b(detail-title|article-title)\b'))
        if title_element:
             for span in title_element.find_all('span', class_='comment'):
                 span.decompose()
             data["title"] = title_element.get_text(strip=True)

        # Mô tả
        description_element = soup.find("h2", class_=re.compile(r'\b(detail-sapo|sapo)\b'))
        if description_element: data["description"] = description_element.get_text(strip=True)

        # Tác giả
        author_div = soup.find("div", class_="author-info")
        if author_div:
            name_element = author_div.find("a", class_="name")
            if name_element:
                author_text = name_element.get_text(strip=True)
                more_authors_span = author_div.find("span", class_="morenameauthor")
                if more_authors_span:
                    author_text += " " + more_authors_span.get_text(strip=True)
                data["author"] = author_text
            elif author_div.get_text(strip=True):
                 author_name = author_div.get_text(strip=True)
                 if "ngày" in author_name.lower():
                      author_name = author_name.split("ngày")[0].strip()
                 data["author"] = author_name

        # Nội dung
        content_div = soup.find("div", class_=re.compile(r'\b(detail-content|content)\b'))
        if content_div:
            elements_to_remove_selectors = [
                '.VCSortableInPreviewMode[type="RelatedNewsBox"]', '.kbwscwl-relatedbox',
                '[type="RelatedOneNews"]', 'div[id^="ObjectBoxContent_"]',
                'figure.VCSortableInPreviewMode', 'figure', 'table', 'video',
                '.image-cms', '.pic', '.video-cms', '.VCSortableInPreviewMode[type="Photo"]',
                '.quangcao', '.qcadt', 'div[id^="ads"]', 'div[class^="ads"]', '#InreadPc',
                '#sticky-player', '.wrapper-sticky-share', '.banner-contain', '.sticky-left',
                '.social-like-share','.share-tool-item', '.like-share-top', '.like-share-bottom',
                'script', 'style', 'noscript', 'iframe',
                'p > strong:contains("TTO")', 'p > em:contains("TTO")', '.author',
                'h4.related-news__title', '.readmore-body-box', '.source'
            ]
            for selector in elements_to_remove_selectors:
                for unwanted in content_div.select(selector):
                     unwanted.decompose()

            main_text_elements = content_div.find_all(['p', 'h2', 'h3'], recursive=True)
            content_parts = []
            for element in main_text_elements:
                 if element.name == 'h2' and element.get('class') and ('detail-sapo' in element.get('class') or 'sapo' in element.get('class')):
                      continue
                 if element.find_parent('figcaption') or (element.get('class') and 'PhotoCMS_Caption' in element.get('class')):
                     continue

                 text = element.get_text(separator='\n', strip=True)
                 if text and len(text) > 10 and not text.startswith(("Ảnh:", "Đồ họa:", "Nguồn:", "*", ">>", "+")):
                      text = re.sub(r'\n\s*\n', '\n\n', text).strip()
                      content_parts.append(text)

            clean_content = "\n\n".join(content_parts)

            if not clean_content or len(clean_content) < 50:
                 clean_content_fallback = content_div.get_text(separator='\n', strip=True)
                 clean_content_fallback = re.sub(r'\n\s*\n', '\n\n', clean_content_fallback).strip()
                 if len(clean_content_fallback) > len(clean_content) + 100:
                     clean_content = clean_content_fallback

            data["content"] = clean_content.strip()

        return data

    except Exception as e:
        print(f"   Lỗi khi xử lý chi tiết bài báo {url}: {e}")
        return data

# --- Hàm tìm link Tuổi Trẻ (Sử dụng cách khởi tạo Selenium cũ) ---
def crawl_tuoitre_links(category, number_target=600):
    """Thu thập link bài báo từ các trang danh mục Tuổi Trẻ bằng Selenium (dựa vào PATH)."""
    base_url = f"https://tuoitre.vn/{category}.htm"
    links = set()
    max_clicks = 60
    clicks = 0
    max_retries = 3
    retries = 0

    # Cấu hình Selenium Options
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
    chrome_options.add_argument("--disable-notifications")
    # chrome_options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2}) # Tắt ảnh nếu muốn

    driver = None
    try:
        # === SỬ DỤNG CÁCH KHỞI TẠO CŨ ===
        print("Đang thử khởi tạo WebDriver (dựa vào PATH)...")
        # Selenium sẽ tự tìm chromedriver trong các thư mục thuộc PATH
        driver = webdriver.Chrome(options=chrome_options)
        print("Khởi tạo WebDriver thành công.")
        # === KẾT THÚC KHỞI TẠO CŨ ===

        print(f"Bắt đầu thu thập link Tuổi Trẻ: {base_url} (Mục tiêu: {number_target})")
        driver.get(base_url)
        WebDriverWait(driver, 20).until(
             EC.presence_of_element_located((By.CSS_SELECTOR, 'h3.box-title-text a, h3.title-news a'))
        )
        time.sleep(2)

        last_height = driver.execute_script("return document.body.scrollHeight")

        # --- Vòng lặp thu thập link ---
        while len(links) < number_target and clicks < max_clicks:
            initial_link_count = len(links)

            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(0.5)

            article_elements = driver.find_elements(By.CSS_SELECTOR, 'h3.box-title-text a, h3.title-news a')
            print(f"   Trang hiện tại có {len(article_elements)} phần tử link tiềm năng.")

            for element in article_elements:
                try:
                    link = element.get_attribute('href')
                    if link and isinstance(link, str) and \
                       link.startswith('https://tuoitre.vn/') and \
                       link.endswith(".htm") and \
                       "?page=" not in link and \
                       link not in links:
                        if not re.search(r'/(video|infographics|podcast|quiz|longform|e-magazine|livestream)/', link):
                            links.add(link)
                            if len(links) >= number_target: break
                except Exception:
                    pass

            print(f"   => Đã thu thập: {len(links)} / {number_target} link duy nhất cho [{category}].")

            if len(links) >= number_target:
                print(f"   Đã đạt mục tiêu {number_target} link cho danh mục [{category}].")
                break

            # --- Xử lý nút "Xem thêm" hoặc cuộn trang ---
            try:
                load_more_button = WebDriverWait(driver, 8).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.btn-readmore.view-more"))
                )
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", load_more_button)
                time.sleep(1)
                print(f"   Đang thử nhấp 'Xem thêm' lần thứ {clicks + 1}/{max_clicks}...")
                driver.execute_script("arguments[0].click();", load_more_button)
                clicks += 1
                print(f"   Đã nhấp 'Xem thêm'. Đang chờ nội dung mới tải...")
                time.sleep(5)
                retries = 0
                last_height = driver.execute_script("return document.body.scrollHeight")

            except TimeoutException:
                print("   Không tìm thấy nút 'Xem thêm' hoặc nút không thể nhấp. Thử cuộn trang...")
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(3)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    print("   Đã cuộn đến cuối trang và không có nội dung mới. Dừng.")
                    break
                last_height = new_height

            except ElementClickInterceptedException:
                 print("   Nhấp 'Xem thêm' bị chặn. Thử cuộn và thử lại...")
                 driver.execute_script("window.scrollBy(0, 400);")
                 time.sleep(2)
                 retries += 1
                 if retries >= max_retries:
                     print(f"   Không thể nhấp 'Xem thêm' sau {max_retries} lần thử lại. Dừng danh mục này.")
                     break
                 continue

            except Exception as e_click:
                print(f"   Lỗi không mong muốn khi click/cuộn: {e_click}. Dừng danh mục này.")
                break

            if len(links) == initial_link_count and clicks > 0 and retries == 0:
                 if clicks > 2:
                      print("   Không có link mới được thêm sau vài lần nhấp 'Xem thêm'. Có thể đã hết bài. Dừng.")
                      break

    # --- Xử lý lỗi chung của Selenium ---
    except WebDriverException as e_selenium: # Bắt lỗi cụ thể hơn của Selenium
        print(f"!!! Lỗi WebDriver khi xử lý danh mục [{category}]: {e_selenium}")
        if "net::ERR_NAME_NOT_RESOLVED" in str(e_selenium):
            print("   Lỗi mạng hoặc không thể phân giải tên miền. Kiểm tra kết nối internet.")
        elif "session deleted because of page crash" in str(e_selenium) or \
             "target window already closed" in str(e_selenium):
            print("   Trình duyệt bị crash hoặc đóng đột ngột.")
        else:
            print("   Lỗi này có thể do chromedriver không tìm thấy trong PATH hoặc phiên bản không tương thích.")
            print("   Vui lòng kiểm tra lại cài đặt chromedriver và biến môi trường PATH.")
        # Không cần return ở đây vì finally sẽ chạy
    except Exception as e_main_selenium: # Bắt các lỗi khác
        print(f"!!! Lỗi không mong muốn trong quá trình Selenium cho danh mục [{category}]: {e_main_selenium}")
    finally:
        if driver:
            print(f"Đang đóng trình duyệt Selenium cho danh mục [{category}]...")
            try:
                driver.quit()
            except Exception as e_quit:
                print(f"   Lỗi khi đóng trình duyệt: {e_quit}")

    print(f"--- Hoàn tất tìm kiếm link cho [{category}]. Tìm thấy tổng cộng {len(links)} link duy nhất. ---")
    return links


# --- Thực thi chính ---
def main():
    tuoitre_categories = [
        "kinh-doanh", "phap-luat", "the-thao", "suc-khoe",
        "giai-tri", "khoa-hoc", "the-gioi", "giao-duc", "du-lich"
    ]
    number_target_per_category = 600
    all_articles_data = []
    collected_links = set()

    print("\n" + "="*10 + f" BẮT ĐẦU THU THẬP DỮ LIỆU TUỔI TRẺ (Mục tiêu: {number_target_per_category} bài/danh mục) " + "="*10)
    overall_start_time = time.time()

    for category in tuoitre_categories:
        print(f"\n===>>> Đang xử lý Danh mục: [{category}] <<<===")
        category_start_time = time.time()
        articles_added_in_category = 0

        # --- Bước 1: Tìm kiếm Links ---
        article_links_found = crawl_tuoitre_links(category, number_target_per_category)

        # --- Bước 2: Trích xuất chi tiết ---
        if article_links_found:
            print(f"\n---> Bắt đầu trích xuất chi tiết cho {len(article_links_found)} link tìm thấy trong [{category}]...")
            links_to_process = list(article_links_found)

            for i, link in enumerate(links_to_process):
                print(f"   [{category} - {i+1}/{len(links_to_process)}] ", end="")

                if link in collected_links:
                    print(f"Link đã xử lý: {link}. Bỏ qua.")
                    continue

                article_details = extract_tuoitre_article_details(link, category)

                if article_details:
                    if article_details["title"] != "Không tìm thấy tiêu đề" and \
                       article_details["content"] != "Không tìm thấy nội dung" and \
                       len(article_details["content"]) > 100:
                        all_articles_data.append(article_details)
                        collected_links.add(link)
                        articles_added_in_category += 1
                        print(f"   => OK. (Đã thêm {articles_added_in_category} bài mới cho danh mục)")
                    else:
                        print(f"   Bài báo không hợp lệ (thiếu TT/ND hoặc ND quá ngắn): {link}. Bỏ qua.")
                else:
                    print(f"   Trích xuất thất bại: {link}. Bỏ qua.")

                time.sleep(0.5)
            print(f"---> Hoàn tất trích xuất cho [{category}]. Đã thêm {articles_added_in_category} bài báo mới.")
        else:
            # Kiểm tra xem hàm crawl_tuoitre_links có trả về set rỗng do lỗi WebDriver không
            # (Lỗi này đã được in ra trong hàm crawl_tuoitre_links)
            if not article_links_found: # Chỉ in thêm nếu thực sự không tìm thấy link (không phải do lỗi WebDriver)
                 print(f"!!! Không tìm thấy link nào trên trang hoặc đã dừng sớm cho danh mục: [{category}]")


        category_end_time = time.time()
        print(f"Thời gian xử lý danh mục [{category}]: {category_end_time - category_start_time:.2f} giây.")
        print("-" * 50)

    overall_end_time = time.time()
    print("\n" + "="*25)
    print(f"*** HOÀN TẤT TOÀN BỘ QUÁ TRÌNH THU THẬP DỮ LIỆU ***")
    print(f"*** Tổng số bài báo duy nhất đã trích xuất và hợp lệ: {len(all_articles_data)} ***")
    total_minutes = (overall_end_time - overall_start_time) / 60
    print(f"Tổng thời gian thực thi: {total_minutes:.2f} phút")
    print("="*25)

    # --- Bước 3: Lưu dữ liệu vào file CSV ---
    if all_articles_data:
        csv_filepath = '/content/drive/MyDrive/FILE/Năm 3/Khoa học dữ liệu/Nhóm 09 - PTDL Báo điện tử/tuoitre_articles_data.csv' # Đổi tên file
        fieldnames = ['category', 'link', 'title', 'author', 'description', 'content']

        print(f"\nĐang lưu dữ liệu vào file CSV: {csv_filepath} ...")
        try:
            with open(csv_filepath, 'w', newline='', encoding='utf-8-sig') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
                writer.writeheader()
                writer.writerows(all_articles_data)
            print(f"===> Đã lưu dữ liệu thành công vào {csv_filepath}")
        except IOError as e:
            print(f"\n!!! Lỗi IO khi lưu file CSV: {e}")
        except Exception as e:
            print(f"\n!!! Lỗi không xác định khi lưu file CSV: {e}")
    else:
        print("\nKhông có dữ liệu bài báo nào hợp lệ để lưu vào file CSV.")

if __name__ == "__main__":
    main()

# Tổng hợp thành file crawl duy nhất

In [ ]:
import pandas as pd

# Đường dẫn đến 3 file CSV (thay đổi theo tên file thực tế của bạn)
file1 = '/content/drive/MyDrive/FILE/Năm 3/Khoa học dữ liệu/Nhóm 09 - PTDL Báo điện tử/vnexpress_all_articles.csv'
file2 = '/content/drive/MyDrive/FILE/Năm 3/Khoa học dữ liệu/Nhóm 09 - PTDL Báo điện tử/dantri_articles_data.csv'
file3 = '/content/drive/MyDrive/FILE/Năm 3/Khoa học dữ liệu/Nhóm 09 - PTDL Báo điện tử/tuoitre_articles_data.csv'

# Đọc 3 file CSV
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

# Gộp 3 DataFrame thành 1 bằng cách nối theo chiều dọc
df_combined = pd.concat([df1, df2, df3], ignore_index=True)

# Kiểm tra dữ liệu sau khi gộp
print("Số dòng của file gộp:", len(df_combined))
print("Thông tin cơ bản của file gộp:")
print(df_combined.info())
print("\nDữ liệu mẫu (5 dòng đầu):")
print(df_combined.head())

# Lưu file gộp thành CSV mới
output_file = '/content/drive/MyDrive/FILE/Năm 3/Khoa học dữ liệu/Nhóm 09 - PTDL Báo điện tử/crawl_data_articles.csv'
df_combined.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"Đã gộp 3 file và lưu vào: {output_file}")